In [1]:
import scipy.io as sio
import numpy as np
import cv2
import random
import colorsys

In [3]:
from matplotlib import pyplot as plt

In [4]:
MPII_MAT = r"/home/dongjai/SDevice2/mpii_human_pose_label/mpii_human_pose_v1_u12_1.mat"

MPII_IMG = r"/home/dongjai/Code/PyTorch-YOLOv3/data/custom/images"

T=sio.loadmat(MPII_MAT, squeeze_me=True, struct_as_record=False)
M=T['RELEASE']

In [5]:
#Get all annotations list
annots=M.annolist

In [6]:
single_person_id = M.single_person

In [7]:
#There are 18079 items in M.img_train which equal to 1, approximatelly 72% for training. Total 24987 items.
is_train=M.img_train

In [8]:
def check_instance(data, data_type):
    if isinstance(data, data_type): 
        data = [data]
    return data

In [9]:
def get_bbox_hw(x1,y1,x2,y2):
    '''
    left top and right buttom point in a rectangle
    '''
    diff_x = abs(x2 - x1)
    diff_y = abs(y2 - y1)
    diff = min(diff_x, diff_y)
    bbox_w = diff / 2
    bbox_h = bbox_w
    return bbox_w, bbox_h

In [10]:
def vis_bbox(img, bboxes, height, width, label):
    num_classes = len(label) + 1 # plus one is the head label
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)
    
    for i, bbox in enumerate(bboxes):
        xc = int(bbox[1] * width)
        yc = int(bbox[2] * height)
        w = int(bbox[3] * width / 2)
        h = int(bbox[4] * height / 2)
        c1 = (xc - w, yc - h)
        c2 = (xc + w, yc + h )
        fontScale = 0.5
        class_ind = bbox[0]
        bbox_color = colors[class_ind]
        bbox_thick = int(0.6 * (height + width) / 600)
        cv2.rectangle(img, c1, c2, bbox_color, bbox_thick)
    
    return img

In [11]:
def get_points(points):
#     point_list = []
    point_dict = {}
    points = check_instance(points, sio.matlab.mio5_params.mat_struct)
#     print("points: ",points)
    for point in points:
        try:
            is_visible = int(point.is_visible)
        except:
            continue
        if is_visible not in [0,1]:
            is_visible = 0
#         point_list.append([point.id,point.x,point.y,is_visible])
        point_dict[point.id] = [point.x, point.y, is_visible]
    return point_dict

In [12]:
def get_labels(keypoints_list, points_dict, bbox_w, bbox_h, img_width, img_height):
    '''
    Get lists of bbox label from a points_dictionary
    OUTPUT: [[cls, xc, yc, w, h], [...], ..., [...]]
    '''
    output = []
    for i, kp in enumerate(keypoints_list):
        if kp in points_dict:
            [x, y, is_visible] = points_dict[kp]
            if is_visible == 1:
                xc = x / img_width
                yc = y / img_height
                w = bbox_w / img_width
                h = bbox_h / img_height
                output.append([i, xc, yc, w, h])
    return output

In [13]:
def get_labels_dict(annots, is_train, keypoints_list):
    label_dict = {}#THE OUTPUT
    for annot_id, annot in enumerate(annots):
        is_train_flag = is_train[annot_id] # train or val_flag:1, test_flag:0, in MPii test data is not be labeled.
        if is_train_flag == 1: 
            img_name = annot.image.name
#             print(img_name)
            try: #some images are missing
                img = plt.imread(MPII_IMG+"/"+img_name)
            except:
                print("could not open the image."+img_name)
                continue
            img_height, img_width = img.shape[0], img.shape[1]

            if 'annorect' in dir(annot):
                rects = annot.annorect #There may have several rectangels in one image to represent several people.
                #Some rects values are only have one person, 'rects' is not a list.
                rects = check_instance(rects, sio.matlab.mio5_params.mat_struct)
                rects = np.array(rects)
                label_lists = []
                for rect in rects:# rects:[[rect1, rect2, rect3, ..., rectn]] #covert all mat_struct to list data style.
                    #get head bbox x1 y1 x2 y2
                    try:
                        head_x1,head_y1,head_x2,head_y2 = rect.x1,rect.y1,rect.x2,rect.y2
                    except:
                        print("no head"+str(is_train_flag)+" "+img_name)
                        continue
                    #get keypoints
                    try:
                        points = rect.annopoints.point
                    except:
                        print("no points"+str(is_train_flag)+" "+img_name)
                        continue
                    #put keypoints into a dict
                    points_dict = get_points(points)
                    #Generate the bbox label for each joint and head
                    #1.set the keypoints list you want to generate bbox label
                    #2.set the bbox width and height
                    bbox_w, bbox_h = get_bbox_hw(head_x1,head_y1,head_x2,head_y2)
                    #3.generate the label:[cls, xc, yc, w, h] normalized by img_width and img_height
                    label_list = get_labels(keypoints_list, points_dict, bbox_w, bbox_h, img_width, img_height)
                    #4.put into a label list
                    for item in label_list:
                        label_lists.append(item)
                    #head
                    head_w = head_x2 - head_x1
                    head_h = head_y2 - head_y1
                    head_xc = (head_x2 - head_w / 2)
                    head_yc = (head_y2 - head_h / 2)
                    label_lists.append([len(keypoints_list), head_xc/img_width, head_yc/img_height, head_w/img_width, head_h/img_height])
                #5.put into a dict to store the label and image name, like: dict{img name:[label]}  
#                 print(label_lists)
#                 draw the bboxes of joints
#                 pic = vis_bbox(img, label_lists, img_height, img_width, keypoints_list)
#                 plt.imshow(pic)
#                 plt.show()
                if label_lists:#pass the empty labels like []
                    label_dict[img_name] = label_lists
    return label_dict


In [14]:
keypoints_list = [0, 1, 4, 5, 10, 11, 14, 15]
label_dict = get_labels_dict(annots, is_train, keypoints_list)

could not open the image.040348287.jpg
could not open the image.013401523.jpg
could not open the image.002878268.jpg
no points1 090903252.jpg
no points1 059897851.jpg
no points1 007049195.jpg
no points1 032783820.jpg
no points1 012525123.jpg
no points1 084820313.jpg
no points1 004929966.jpg
no points1 080117150.jpg
no points1 025130769.jpg
no points1 009695139.jpg
no points1 056338374.jpg
no points1 011605346.jpg
no points1 093296255.jpg
no points1 043307451.jpg
no points1 045256450.jpg
no points1 083956671.jpg
no points1 009876905.jpg
no points1 041591653.jpg
no points1 030538148.jpg
no points1 030538148.jpg
no points1 062188196.jpg
no points1 023769876.jpg
no points1 048169984.jpg
no points1 048169984.jpg
no points1 040065785.jpg
no points1 082112363.jpg
no points1 088969373.jpg
no points1 060868753.jpg
no points1 037186012.jpg
no points1 099861143.jpg
no points1 072810449.jpg
no points1 087202973.jpg
no points1 075765762.jpg
no points1 006504596.jpg
no points1 070173104.jpg
no point

In [15]:
# Only for debugging
# for img, label in label_dict.items():
#     print(img)
#     print(label)

In [16]:
def write_label(folder, name, text):
    txt = open(folder+'/'+name[:-4]+'.txt','a')
    for n in text:
        wt_str = ""
        for item_ in n:
            wt_str = wt_str + str(item_) + " "
        txt.write(wt_str[:-1]+"\r")
    txt.close()

In [17]:
IMG_train_txt = r"./train.txt" #sotre the image path
IMG_valid_txt = r"./valid.txt"
LABEL=r"/home/dongjai/Code/PyTorch-YOLOv3/data/custom/labels" #sotre all labels txt file

In [18]:
train_txt = open(IMG_train_txt,'a')
valid_txt = open(IMG_valid_txt,'a')

In [19]:
for_train = 0
len_data = len(label_dict)
for img_name, label_list in label_dict.items():
    if for_train < len_data*0.8:
        #open train.txt to store the img path
        train_txt.write(MPII_IMG+"/"+img_name+"\r")
        for_train += 1
        write_label(LABEL, img_name, label_list)
    else:
        #open val.txt to store the img path
        valid_txt.write(MPII_IMG+"/"+img_name+"\r")
        write_label(LABEL, img_name, label_list)
train_txt.close()
valid_txt.close()
